In [30]:
######################################     Modules     #######################################
import sys
from time import process_time
import os
import pickle

sys.path.append('../.')
from verbose_module import routing_instances as verb
sys.path.append('../../../.')
import pIRPgym


computer_name = input("Running experiment on mac? [Y/n]")
if computer_name == '': 
    path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
    experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/'
else: 
    path = 'C:/Users/jm.betancourt/Documents/Research/pIRPgym/'
    experiments_path = 'G:/Mi unidad/Research/Supply Chain Analytics/Experiments/Classic Instances/'


instances = dict()
instances['Li'] = [i for i in os.listdir(path+'/pIRPgym/Instances/CVRP Instances/dCVRP/Li') if i[-3:]=='vrp']
instances['Golden'] = [i for i in os.listdir(path+'/pIRPgym/Instances/CVRP Instances/dCVRP/Golden') if i[-3:]=='vrp']
instances['Uchoa'] = [i for i in os.listdir(path+'pIRPgym/Instances/CVRP Instances/CVRP/Uchoa') if i[-3:]=='vrp']
instances['Li'].sort();instances['Golden'].sort()
instances['Uchoa'].sort();instances['Uchoa'] = instances['Uchoa'][1:]+[instances['Uchoa'][0]]

Policies = ['NN','RCL','GA']

verbose = True

# dCVRP Instances

In [33]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)

for inst_set in ['Li','Golden']:
    if verbose: verb.print_head(Policies,inst_set,show_gap=True)
    
    for instance in instances[inst_set]:
        purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
        if verbose: string = verb.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
        
        for policy in Policies:
            end=False
            if policy==Policies[-1]: end=True
            # print(f'{inst_set}/{policy}/{instance[:-4]}.pkl')
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
            
            if policy in ['NN','GA']:
                if verbose: string = verb.print_routing_update(string,performance[1],len(performance[0]),
                                                                                 performance[3],True,benchmark,end=end)
            elif policy=='RCL':
                if verbose: string = verb.print_routing_update(string,performance[0],performance[1],
                                                                                 performance[2],True,benchmark,intervals=performance[3],end=end)
    
    print('\n')

********************************************************  Golden set Instances  ********************************************************
--------|-----------------------|	  NN 	 	|	  	  	 	RCL 	 	 	|	  GA 	 	|
Inst	|   M 	  Veh	 Obj	| t(s)	 #Veh 	 gap 	| t(s)	 #Veh 	 mean 	median	 stdev	 min	 max	| t(s)	 #Veh 	 gap 	|
--------------------------------------------------------------------------------------------------------------------------------------------
Go 1 	|  240	  9 	5623 	| 0.00	  14	 32.18	| 0.03	  13.9	 35.84	 33.82 	8.86	  27.02	 68.95	| 34.9	  13	 32.34	|
Go 2 	|  320	  10 	8405 	| 0.02	  15	 33.93	| 0.03	  15.4	 41.52	 36.38 	14.85	  28.49	 87.19	| 126.0	  14	 42.7	|
Go 3 	|  400	  9 	10998 	| 0.00	  15	 39.93	| 0.05	  15.1	 47.81	 36.8 	19.74	  30.95	 102.04	| 168.0	  15	 52.53	|
Go 4 	|  480	  10 	13589 	| 0.02	  13	 31.83	| 0.07	  13.3	 33.77	 30.16 	11.18	  23.46	 76.41	| 201.0	  14	 56.45	|
Go 5 	|  200	  5 	6461 	| 0.00	  6	 49.44	| 0.02	  6.0	 38.1	 34.54 	12.36	  1

### GA 

In [27]:
POPULATION_SIZES = [250,750,1000,2500]
ELITE_PROPORTIONS = [0.05,0.15,0.3]
MUTATION_RATES = [0.25,0.5,0.75]

combinations = [(p,e,m) for p in POPULATION_SIZES for e in ELITE_PROPORTIONS for m in MUTATION_RATES]
per = {com:list() for com in combinations}

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)


for instance in instances['Li'][0:8]:
    purchase,benchmark = inst_gen.upload_CVRP_instance('Li',instance)

    # for (p,e,m) in combinations:
    with open(experiments_path+f'Li/GA/{instance[:-4]}_{p}_{e}_{m}.pkl', 'rb') as file:
        # Use pickle.dump to serialize and save the dictionary to the file
        performance = pickle.load(file)

    per[(p,e,m)].append(round((performance[1]-benchmark[0])/benchmark[0],2))
        # print(f'{p}_{e}_{m} \t',round((performance[1]-benchmark[0])/benchmark[0],2))
        # print(f'{p}_{e}_{m} \t',performance[1])

per = {com:sum(per[com])/len(per[com]) for com in combinations}

FileNotFoundError: [Errno 2] No such file or directory: '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/Li/GA/Li_21_250_0.05_0.25.pkl'

In [23]:
for com in combinations:
    print(com, '\t', per[com])

(250, 0.05, 0.25) 	 0.7749999999999999
(250, 0.05, 0.5) 	 0.78125
(250, 0.05, 0.75) 	 0.7737499999999999
(250, 0.15, 0.25) 	 0.77
(250, 0.15, 0.5) 	 0.78
(250, 0.15, 0.75) 	 0.7775000000000001
(250, 0.3, 0.25) 	 0.76875
(250, 0.3, 0.5) 	 0.7875000000000001
(250, 0.3, 0.75) 	 0.7712500000000002
(750, 0.05, 0.25) 	 0.7675000000000001
(750, 0.05, 0.5) 	 0.76375
(750, 0.05, 0.75) 	 0.7575
(750, 0.15, 0.25) 	 0.7662499999999999
(750, 0.15, 0.5) 	 0.7562500000000001
(750, 0.15, 0.75) 	 0.77
(750, 0.3, 0.25) 	 0.7575
(750, 0.3, 0.5) 	 0.7725000000000001
(750, 0.3, 0.75) 	 0.7675000000000001
(1000, 0.05, 0.25) 	 0.76625
(1000, 0.05, 0.5) 	 0.74625
(1000, 0.05, 0.75) 	 0.7425
(1000, 0.15, 0.25) 	 0.73875
(1000, 0.15, 0.5) 	 0.7525
(1000, 0.15, 0.75) 	 0.925
(1000, 0.3, 0.25) 	 0.755
(1000, 0.3, 0.5) 	 0.7675000000000001
(1000, 0.3, 0.75) 	 0.76875
(2500, 0.05, 0.25) 	 0.75125
(2500, 0.05, 0.5) 	 0.7374999999999999
(2500, 0.05, 0.75) 	 0.7474999999999999
(2500, 0.15, 0.25) 	 0.7575000000000001
(

# CVRP Instances

### exo-alpha

In [5]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.routing_instances.print_head(Policies,inst_set,show_gap=True)

for instance in instances[inst_set][1:]:
    purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
    if verbose: string = verb.routing_instances.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
    
    for policy in Policies:
        end=False
        if policy==Policies[-1]: end=True

        if policy == 'RCL':
            with open(experiments_path+f'{inst_set}/{policy}/Random/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        else:
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        
        if policy=='NN':
            if verbose: string = verb.routing_instances.print_routing_update(string,performance[1],len(performance[0]),
                                                                                performance[3],True,benchmark,end=end)
        elif policy=='RCL':
            if verbose: 
                string = verb.routing_instances.print_routing_update(string,performance[0],performance[1],
                                                                                performance[2],True,benchmark,intervals=performance[3],end=end)
            RCL_averge_gap+=((performance[0]-benchmark[0])/benchmark[0])/100

print('RCL average gap:', round(RCL_averge_gap,4)*100)

AttributeError: type object 'routing_instances' has no attribute 'routing_instances'

In [6]:
def create_node_range_library(instance_names):
    node_range_library = {}

    for instance_name in instance_names:
        # Extract the number of nodes from the instance name
        nodes = int(instance_name.split('-n')[1].split('-')[0])

        # Determine the range tuple
        node_range = ((nodes // 100) * 100, ((nodes // 100) * 100) + 99)

        # Add the instance to the corresponding range in the dictionary
        if node_range not in node_range_library:
            node_range_library[node_range] = [instance_name]
        else:
            node_range_library[node_range].append(instance_name)

    return node_range_library

# Example usage:
# Call the function to create the library
instance_sizes = create_node_range_library(instances['Uchoa'])

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_comparison_head(Policies,inst_set,show_gap=True)

for interval,inst_list in instance_sizes.items():

    if verbose: string = verb.print_comparison_inst(interval,len(inst_list))
    NN_gap = 0
    NN_time = 0
    RCL_gap = 0
    RCL_time = 0
    RCL_min = 0
    RCL_max = 0

    for instance in inst_list:
        purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

        with open(experiments_path+f'Uchoa/NN/{instance[:-4]}.pkl', 'rb') as file:
            NN_performance = pickle.load(file)
            NN_gap += (((NN_performance[1]-benchmark[0])/benchmark[0])/len(inst_list))
            NN_time += NN_performance[3]/len(inst_set)


        with open(experiments_path+f'Uchoa/RCL/Random/{instance[:-4]}.pkl', 'rb') as file:
            RCL_performance = pickle.load(file)
            RCL_gap += ((RCL_performance[0]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_time += RCL_performance[2]/len(inst_list)/len(inst_list)
            RCL_min += ((RCL_performance[3][1]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_max += ((RCL_performance[3][2]-benchmark[0])/benchmark[0])/len(inst_list)
    

    verb.print_routing_comparison_update(string,NN_gap,NN_time,RCL_gap,RCL_time,RCL_min,RCL_max)


****************************  Uchoa set Instances  *****************************
----------------|	NN 	|	     RCL 	 	|
Sizes	  Num 	| t(s) 	   gap 	| t(s)	  mean	 median 	   min	   max	|
-----------------------------------------------------------------


FileNotFoundError: [Errno 2] No such file or directory: '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Classic Instances/Uchoa/RCL/Random/X-n101-k25.pkl'

### intra-alpha

In [ ]:
# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_head(Policies,inst_set,show_gap=True)

for instance in instances[inst_set][1:]:
    purchase,benchmark = inst_gen.upload_CVRP_instance(inst_set,instance)
    if verbose: string = verb.print_inst(inst_set,instance,inst_gen.M,benchmark[1],benchmark[0])
    
    for policy in Policies:
        end=False
        if policy==Policies[-1]: end=True

        if policy == 'RCL':
            with open(experiments_path+f'{inst_set}/{policy}/Adaptative/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        else:
            with open(experiments_path+f'{inst_set}/{policy}/{instance[:-4]}.pkl', 'rb') as file:
                # Use pickle.dump to serialize and save the dictionary to the file
                performance = pickle.load(file)
        
        if policy=='NN':
            if verbose: string = verb.print_routing_update(string,performance[1],len(performance[0]),
                                                                                performance[3],True,benchmark,end=end)
        elif policy=='RCL':
            if verbose: string = verb.print_routing_update(string,performance[0],performance[1],
                                                                                performance[2],True,benchmark,intervals=performance[3],end=end)
            RCL_averge_gap+=((performance[0]-benchmark[0])/benchmark[0])/100

print('RCL average gap', round(RCL_averge_gap,4)*100)

In [ ]:
def create_node_range_library(instance_names):
    node_range_library = {}

    for instance_name in instance_names:
        # Extract the number of nodes from the instance name
        nodes = int(instance_name.split('-n')[1].split('-')[0])

        # Determine the range tuple
        node_range = ((nodes // 100) * 100, ((nodes // 100) * 100) + 99)

        # Add the instance to the corresponding range in the dictionary
        if node_range not in node_range_library:
            node_range_library[node_range] = [instance_name]
        else:
            node_range_library[node_range].append(instance_name)

    return node_range_library

# Example usage:
# Call the function to create the library
instance_sizes = create_node_range_library(instances['Uchoa'])

# Creating instance generator object
inst_gen = pIRPgym.instance_generator(False,False,False,False)
inst_set='Uchoa'
RCL_averge_gap = 0

if verbose: verb.print_comparison_head(Policies,inst_set,show_gap=True)

for interval,inst_list in instance_sizes.items():

    if verbose: string = verb.print_comparison_inst(interval,len(inst_list))
    NN_gap = 0
    NN_time = 0
    RCL_gap = 0
    RCL_time = 0
    RCL_min = 0
    RCL_max = 0

    for instance in inst_list:
        purchase,benchmark = inst_gen.upload_CVRP_instance('Uchoa',instance)

        with open(experiments_path+f'Uchoa/NN/{instance[:-4]}.pkl', 'rb') as file:
            NN_performance = pickle.load(file)
            NN_gap += (((NN_performance[1]-benchmark[0])/benchmark[0])/len(inst_list))
            NN_time += NN_performance[3]/len(inst_set)


        with open(experiments_path+f'Uchoa/RCL/Adaptative/{instance[:-4]}.pkl', 'rb') as file:
            RCL_performance = pickle.load(file)
            RCL_gap += ((RCL_performance[0]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_time += RCL_performance[2]/len(inst_list)/len(inst_list)
            RCL_min += ((RCL_performance[3][1]-benchmark[0])/benchmark[0])/len(inst_list)
            RCL_max += ((RCL_performance[3][2]-benchmark[0])/benchmark[0])/len(inst_list)
    

    verb.print_routing_comparison_update(string,NN_gap,NN_time,RCL_gap,RCL_time,RCL_min,RCL_max)
